#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [2]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [4]:
embeddings=OllamaEmbeddings(model="gemma:2b")
db=FAISS.from_documents(docs,embeddings)
db

In [5]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

#### As a Retriever (LLM Chatbot)
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [6]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

Here lower score is better since it's the distance

In [7]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='117e18d4-915d-4bc6-b813-8741f702fd65', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(2937.1094)),
 (Document(id='7a4f7877-d8dc-4e40-9c45-fbbbfa661b23', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed 

In [8]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[0.2929322123527527,
 -2.1362297534942627,
 0.25983577966690063,
 1.0182870626449585,
 0.6563823819160461,
 0.5080406069755554,
 -0.8863821029663086,
 0.1391080617904663,
 -0.09333895891904831,
 -0.7779372930526733,
 1.1261528730392456,
 -0.024418359622359276,
 -1.056193232536316,
 1.0850974321365356,
 0.05699522793292999,
 -1.016982078552246,
 3.0423943996429443,
 1.6945816278457642,
 1.1469627618789673,
 0.7534157037734985,
 0.32203540205955505,
 -0.2788503170013428,
 0.3289756774902344,
 1.3427278995513916,
 0.23575782775878906,
 -0.4106498658657074,
 -1.344303011894226,
 -1.2939201593399048,
 -0.5488048195838928,
 -2.1072607040405273,
 -0.2555670440196991,
 -1.421248435974121,
 1.1993601322174072,
 -0.9413832426071167,
 -0.4193810820579529,
 -0.305507093667984,
 1.8732502460479736,
 0.49941661953926086,
 0.1906089037656784,
 -0.5722184181213379,
 0.37912657856941223,
 0.5510533452033997,
 0.9627256989479065,
 -1.3059675693511963,
 -1.3518562316894531,
 0.6331496238708496,
 0.074756

In [9]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='117e18d4-915d-4bc6-b813-8741f702fd65', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='7a4f7877-d8dc-4e40-9c45-fbbbfa661b23', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. Ther

In [10]:
### Saving And Loading
db.save_local("faiss_index")

In [11]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True) #allow_dangerous_deserialization=True is needed to load the index from a file because the index is stored in a binary format that is not safe to load from untrusted sources
#embeddings already defined as embeddings=OllamaEmbeddings(model="gemma:2b") so we can use it to load the index
docs=new_db.similarity_search(query)

In [12]:
docs

[Document(id='117e18d4-915d-4bc6-b813-8741f702fd65', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='7a4f7877-d8dc-4e40-9c45-fbbbfa661b23', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. Ther